In [1]:
import pandas as pd
import math
import numpy as np
import pickle

In [2]:
# Function to convert geographic coordinates to Cartesian coordinates
def geo_to_cartesian(latitude, longitude):
    # Earth radius in kilometers
    R = 6371.0 
    
    # Convert latitude and longitude from degrees to radians
    lat_rad = math.radians(latitude)
    lon_rad = math.radians(longitude)
    
    # Convert to Cartesian coordinates
    x = R * math.cos(lat_rad) * math.cos(lon_rad)
    y = R * math.cos(lat_rad) * math.sin(lon_rad)
    
    return np.float64(x), np.float64(y)

In [3]:
# Function to convert Cartesian coordinates to geographic coordinates
def cartesian_to_geo(x, y):
    # Earth radius in kilometers
    R = 6371.0 
    
    # Convert to geographic coordinates
    latitude = math.degrees(math.asin(y / R))
    longitude = math.degrees(math.acos(x / (R * math.cos(math.radians(latitude)))))
    
    return latitude, longitude

In [4]:
def get_data_boundaries(data):
    num_points = len(data[0])
    n_users = len(data)
    x_min = min(point[0] for pointlist in data for point in pointlist)
    y_min = min(point[1] for pointlist in data for point in pointlist)
    x_max = max(point[0] for pointlist in data for point in pointlist)
    y_max = max(point[1] for pointlist in data for point in pointlist)


    return x_min, y_min, x_max, y_max, num_points, n_users

In [5]:
#read kaggle dataset dataset
df_kaggle = pd.read_csv('train.csv')

In [6]:
#convert the str to a list. From each list get only the 20 firsts coordinates
df_kaggle['POLYLINE'] = df_kaggle['POLYLINE'].apply(lambda x: eval(x)[:40])

# df_kaggle['POLYLINE'] = df_kaggle['POLYLINE'].apply(lambda x: eval(x))

In [9]:
#remove all rows where the length of the 'POLYLINE' is less than 40
df_kaggle = df_kaggle[df_kaggle['POLYLINE'].apply(lambda x: len(x) == 40)]

In [12]:
len(df_kaggle)

910456

In [13]:
#convert the column POLYLINE from longitude and latitude to cartesian coordinates, inverting the order of the coordinates to x and y
df_kaggle['POLYLINE'] = df_kaggle['POLYLINE'].apply(lambda x: [geo_to_cartesian(x, y) for y, x in x])

In [10]:
#df_kaggle['POLYLINE'].iloc[0]

In [11]:
# calculate the euclidean distance btw (4743.744305381216, 4144.273614650897),(4743.74871634129, 4144.272212746029)
#math.sqrt((4743.744305381216 - 4743.74871634129)**2 + (-719.002784051208 - -718.9912563855992)**2)

In [12]:
# lat, lon = cartesian_to_geo(-2164.4891376291075,4374.242558587631)
# print(lat, lon)
# lat, lon = cartesian_to_geo(-2163.6552191141714,4374.030051225528)
# print(lat, lon)

In [13]:
# math.sqrt((-2164.4891376291075 - -2163.6552191141714)**2 + (4374.242558587631 - 4374.030051225528)**2)

In [14]:
#create a list from the column 'POLYLINE' in df_kanggle
list_kaggle = df_kaggle['POLYLINE'].tolist()

In [15]:
#save the list in a pickl file

with open('taxi_portugal.pkl', 'wb') as f:
    pickle.dump(list_kaggle, f)

In [36]:
#load pickle
with open('taxi_portugal.pkl', 'rb') as f:
    list_kaggle = pickle.load(f)

In [15]:
len(list_kaggle)

910456

In [8]:
#load pickle
# with open('../Taxi_Beijin/cartesian_trajectory_points_60s_170_100_20.pkl', 'rb') as f:
#     list_geo = pickle.load(f)

In [10]:
#len(list_geo[0])

20

In [16]:
#Porto Coordenates - Pontos Extremos
porto_points = [(41.141602, -8.667482),(41.142652, -8.557375),(41.179378, -8.689936),(41.186604, -8.571467)]

#convert all the lon,lat points on porto_points list to cartesian points
porto_points = [geo_to_cartesian(x, y) for x, y in porto_points]


In [17]:
porto_points

[(np.float64(4743.115961958049), np.float64(-723.0440017032093)),
 (np.float64(4744.420736821281), np.float64(-713.9162552355908)),
 (np.float64(4740.0993317758775), np.float64(-724.4850530093972)),
 (np.float64(4741.06403256535), np.float64(-714.6036759099666))]

In [18]:
x_min = min(point[0] for point in porto_points)
x_max = max(point[0] for point in porto_points)
y_min = min(point[1] for point in porto_points)
Y_max = max(point[1] for point in porto_points)

print(x_min, x_max, y_min, Y_max)

4740.0993317758775 4744.420736821281 -724.4850530093972 -713.9162552355908


In [19]:
new_list = []

for tr in list_kaggle:
    outside = False
    for point in tr:
        if point[0] < x_min or point[0] > x_max or point[1] < y_min or point[1] > Y_max:
            outside = True
            break
    if not outside:
        new_list.append(tr)

list_kaggle = new_list

In [20]:
len(list_kaggle)

753388

In [21]:
x_min, y_min, x_max, y_max, num_points, n_users = get_data_boundaries(list_kaggle)

In [22]:
print(x_min, y_min, x_max, y_max, num_points, n_users)

4740.099341911621 -724.4850351942058 4744.420687481553 -713.9164216532397 40 753388


In [23]:
math.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)

11.41795164261344

In [24]:
#save the list in a pickl file

file_name = 'taxi_portugal_{}.pkl'.format(num_points)

with open(file_name, 'wb') as f:
    pickle.dump(list_kaggle, f)